In [1]:
import pandas as pd
import numpy as np
import datetime
import sklearn
import pickle

In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/03-training/experiment_tracking/mlruns/1', creation_time=1738676613877, experiment_id='1', last_update_time=1738676613877, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
train_df = pd.read_csv("./data/green_tripdata_2024-01.csv")

/tmp/ipykernel_7505/3691249756.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("./data/green_tripdata_2024-01.csv")


In [4]:
train_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [5]:
train_df["duration"] = pd.to_datetime(train_df["tpep_dropoff_datetime"]) - pd.to_datetime(train_df["tpep_pickup_datetime"])
train_df["duration"] = train_df["duration"].dt.total_seconds() / 60

In [6]:
train_df = train_df[train_df["duration"] < 60]

In [7]:
train_df.loc[:, "PULocationID"] = train_df["PULocationID"].astype(str)
train_df.loc[:, "DOLocationID"] = train_df["DOLocationID"].astype(str)

/tmp/ipykernel_7505/2614523797.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['186' '140' '236' ... '211' '107' '138']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_df.loc[:, "PULocationID"] = train_df["PULocationID"].astype(str)
/tmp/ipykernel_7505/2614523797.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['79' '236' '79' ... '25' '13' '75']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_df.loc[:, "DOLocationID"] = train_df["DOLocationID"].astype(str)


In [8]:
feature_df = train_df[["PULocationID", "DOLocationID"]].drop_duplicates().reset_index(drop=True)

In [9]:
feature_df

,PULocationID,DOLocationID
0,186,79
1,140,236
2,236,79
3,79,211
4,211,148
...,...,...
24123,62,180
24124,115,115
24125,79,101
24126,106,13


In [10]:
feature_dicts = feature_df.to_dict('records')

In [11]:
feature_dicts

[{'PULocationID': '186', 'DOLocationID': '79'},
 {'PULocationID': '140', 'DOLocationID': '236'},
 {'PULocationID': '236', 'DOLocationID': '79'},
 {'PULocationID': '79', 'DOLocationID': '211'},
 {'PULocationID': '211', 'DOLocationID': '148'},
 {'PULocationID': '148', 'DOLocationID': '141'},
 {'PULocationID': '138', 'DOLocationID': '181'},
 {'PULocationID': '246', 'DOLocationID': '231'},
 {'PULocationID': '161', 'DOLocationID': '261'},
 {'PULocationID': '113', 'DOLocationID': '113'},
 {'PULocationID': '107', 'DOLocationID': '137'},
 {'PULocationID': '158', 'DOLocationID': '246'},
 {'PULocationID': '246', 'DOLocationID': '190'},
 {'PULocationID': '68', 'DOLocationID': '90'},
 {'PULocationID': '90', 'DOLocationID': '68'},
 {'PULocationID': '132', 'DOLocationID': '216'},
 {'PULocationID': '164', 'DOLocationID': '79'},
 {'PULocationID': '237', 'DOLocationID': '237'},
 {'PULocationID': '141', 'DOLocationID': '263'},
 {'PULocationID': '161', 'DOLocationID': '263'},
 {'PULocationID': '263', 'DO

In [12]:
from sklearn.feature_extraction import DictVectorizer

In [13]:
dv = DictVectorizer()

In [14]:
dv.fit_transform(feature_dicts).toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(24128, 520))

In [15]:
dv.get_feature_names_out()

array(['DOLocationID=1', 'DOLocationID=10', 'DOLocationID=100',
       'DOLocationID=101', 'DOLocationID=102', 'DOLocationID=105',
       'DOLocationID=106', 'DOLocationID=107', 'DOLocationID=108',
       'DOLocationID=109', 'DOLocationID=11', 'DOLocationID=111',
       'DOLocationID=112', 'DOLocationID=113', 'DOLocationID=114',
       'DOLocationID=115', 'DOLocationID=116', 'DOLocationID=117',
       'DOLocationID=118', 'DOLocationID=119', 'DOLocationID=12',
       'DOLocationID=120', 'DOLocationID=121', 'DOLocationID=122',
       'DOLocationID=123', 'DOLocationID=124', 'DOLocationID=125',
       'DOLocationID=126', 'DOLocationID=127', 'DOLocationID=128',
       'DOLocationID=129', 'DOLocationID=13', 'DOLocationID=130',
       'DOLocationID=131', 'DOLocationID=132', 'DOLocationID=133',
       'DOLocationID=134', 'DOLocationID=135', 'DOLocationID=136',
       'DOLocationID=137', 'DOLocationID=138', 'DOLocationID=139',
       'DOLocationID=14', 'DOLocationID=140', 'DOLocationID=141',
  

520

Get feature matrix

In [16]:
feature_dicts = train_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [17]:
feature_matrix_train = dv.transform(feature_dicts)

# Training

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [19]:
duration = train_df["duration"].to_numpy()

In [20]:
lr = LinearRegression().fit(feature_matrix_train, duration)

In [21]:
# save model
with open('./models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [22]:
duration_pred = lr.predict(feature_matrix_train)
mse = mean_squared_error(duration, duration_pred)
rmse = np.sqrt(mse)
rmse

np.float64(8.24137308767574)

In [23]:
del train_df

import gc
gc.collect()

8

In [24]:
val_df = pd.read_csv("./data/green_tripdata_2024-02.csv")

/tmp/ipykernel_7505/1637032188.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  val_df = pd.read_csv("./data/green_tripdata_2024-02.csv")


In [25]:
val_df["duration"] = pd.to_datetime(val_df["tpep_dropoff_datetime"]) - pd.to_datetime(val_df["tpep_pickup_datetime"])
val_df["duration"] = val_df["duration"].dt.total_seconds() / 60
val_df = val_df[val_df["duration"] < 60]

val_df.loc[:, "PULocationID"] = val_df["PULocationID"].astype(str)
val_df.loc[:, "DOLocationID"] = val_df["DOLocationID"].astype(str)

/tmp/ipykernel_7505/133451804.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['68' '48' '132' ... '113' '164' '79']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  val_df.loc[:, "PULocationID"] = val_df["PULocationID"].astype(str)
/tmp/ipykernel_7505/133451804.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['236' '243' '261' ... '249' '107' '142']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  val_df.loc[:, "DOLocationID"] = val_df["DOLocationID"].astype(str)


In [26]:
feature_dicts = val_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')
feature_matrix_val = dv.transform(feature_dicts)

val_duration = val_df["duration"].to_numpy()
duration_pred = lr.predict(feature_matrix_val)
mse = mean_squared_error(val_duration, duration_pred)
rmse = np.sqrt(mse)
rmse

np.float64(8.39373788463708)

Train 2nd model

In [27]:
from sklearn.linear_model import Lasso

In [28]:
with mlflow.start_run():
    
    # setting tags for easier filtering
    mlflow.set_tag("developer", "katie")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2024-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2024-02.csv")
    
    alpha = 0.1
    
    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(feature_matrix_train, duration)

    mlflow.log_metric("rmse", rmse)

## Logging hyper-parameters

In [30]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [31]:
train = xgb.DMatrix(feature_matrix_train, label=duration)
valid = xgb.DMatrix(feature_matrix_val, label=val_duration)

In [32]:
# setting objective function
def objective(params):

    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            # on training data
            dtrain=train,
            num_boost_round=1000,
            # use validation set to control the optimisation algorithm
            # minimise error on validation set
            evals=[(valid, "validation")],
            # stop if no improvements for 50 rounds
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
# search space - range for hyperopt to explore the parameters
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    # log uniform space
    'learning_rate': hp.loguniform('learning_rate', -3, 0),   # between exp(-3) and exp(0) - [0.05, 1]
    'reg_alpha': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    # this is a regression problem, so use regression as objective function
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|                                                                    | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/conda/envs/exp-tracking-env/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [15:36:49] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.75191                                                                                     
[1]	validation-rmse:7.92029                                                                                     
[2]	validation-rmse:6.85710                                                                                     
[3]	validation-rmse:6.50380                                                                                     
[4]	validation-rmse:6.36666                                                                                     
[5]	validation-rmse:6.28027                                                                                     
[6]	validation-rmse:5.93903                                                                                     
[7]	validation-rmse:5.90592                                                                                     
[8]	validation-rmse:5.80571                                                                     

In [ ]:
# trained params
params = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), 
    'reg_alpha': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

# using autolog
mlflow.xgboost.autolog()
booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, "validation")],
    early_stopping_rounds=50
)